<a href="https://www.inove.com.ar"><img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center"></a>


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
def get_docs(corpus):
  docs = []
  for e in corpus:
    docs.append(e.split())
  return docs

def get_vocab(corpus):
  vocab = set()
  for e in corpus:
    vocab.update(e.split())
  vocab=np.array(list(vocab))
  return vocab

In [5]:
get_docs(corpus)

[['que', 'dia', 'es', 'hoy'],
 ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'],
 ['martes', 'muchas', 'gracias']]

In [6]:
get_vocab(corpus)

array(['que', 'martes', 'de', 'gracias', 'el', 'hoy', 'dia', 'es',
       'muchas'], dtype='<U7')

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [7]:
def one_hot_encoding(corpus):
  vocab = get_vocab(corpus)
  result = np.zeros(shape=(len(corpus),len(vocab)))
  for i,e in enumerate(corpus):
    result[i]=np.max(vocab.reshape((1,-1)) == np.array(e.split()).reshape((-1,1)),axis=0)
  return result

one_hot_encoding(corpus)

array([[1., 0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 1., 1., 0., 1., 1., 1., 1., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 1.]])

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [8]:
def frequency_vector(corpus):
  vocab = get_vocab(corpus)
  result = np.zeros(shape=(len(corpus),len(vocab)))
  for i,e in enumerate(corpus):
    result[i]=np.sum(vocab.reshape((1,-1)) == np.array(e.split()).reshape((-1,1)),axis=0)
  return result

frequency_vector(corpus)

array([[1., 0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 2., 1., 0., 1., 1., 1., 1., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 1.]])

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [9]:
def tf_idf(corpus):
  vocab = get_vocab(corpus)
  # multiple vocab calculation on tf and one hot. 
  # Inline code to improve performance?
  tf = frequency_vector(corpus) 
  one_hot = one_hot_encoding(corpus)
  idf = np.log10(tf.shape[0]/np.sum(one_hot,axis=0))
  tf_idf = tf * idf
  return tf_idf

tf_idf(corpus)

array([[0.47712125, 0.        , 0.        , 0.        , 0.        ,
        0.17609126, 0.17609126, 0.17609126, 0.        ],
       [0.        , 0.35218252, 0.47712125, 0.        , 0.47712125,
        0.17609126, 0.17609126, 0.17609126, 0.        ],
       [0.        , 0.17609126, 0.        , 0.47712125, 0.        ,
        0.        , 0.        , 0.        , 0.47712125]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [10]:
def doc_order(corpus, index):
  vocab = get_vocab(corpus)
  order_idx = np.zeros(len(corpus))
  tf_table = tf_idf(corpus)

  for i,e in enumerate(corpus):
    order_idx[i] = cosine_similarity(tf_table[index],tf_table[i])

  result = corpus[np.argsort(order_idx)][::-1]

  return result

In [11]:
doc_order(corpus, 1)

array(['martes el dia de hoy es martes', 'que dia es hoy',
       'martes muchas gracias'], dtype='<U30')